<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.sandbox.google.com/github/dvadym/dp/blob/main/dp_accounting/mixture_of_gaussians.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/dvadym/dp/blob/main/dp_accounting/mixture_of_gaussians.ipynb".ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

In [ ]:
!pip install dp-accounting==0.4.3

## Problem setup
Let $\sigma > 0, \alpha_0\in [0, 1], \alpha_2\in [0, 1]$ be fixed numbers.
The goal is to compute discrete PLD with [privacy bucket](https://eprint.iacr.org/2017/1034.pdf) approach for PLD of
Gaussian mixtures, i.e.:

$$PLD(\alpha_0N(0, \sigma^2)+(1-\alpha_0)N(1, \sigma^2) || \alpha_1N(0, \sigma^2)+(1-\alpha_1)N(1, \sigma^2))$$

In [ ]:
#@title Creating PLD for 2 mixture of Gaussians with privacy bucket approach https://eprint.iacr.org/2017/1034.pdf

import numpy as np
from scipy.stats import norm
from dp_accounting.pld import pld_pmf
from dp_accounting.pld import privacy_loss_distribution

def inverse_privacy_loss(x:float | np.ndarray, alpha0: float, alpha1: float, sigma: float)->float | np.ndarray:
  exp_x = np.exp(x)
  nom = exp_x*alpha1 - alpha0
  den = 1-alpha0 - exp_x*(1-alpha1)
  return 0.5 + sigma**2*np.log(nom/den)


def create_pld_pmf(discretization:float,  alpha0: float, alpha1: float, sigma: float, sensitivity:float = 1)->pld_pmf.PLDPmf:
  assert (0 < alpha0 < 1) and (0 < alpha1 < 1) # todo: implement inverse case and =0 or =1
  # If alpha0 < alpha1:
  #   privacy_loss(x) increases so inverse_privacy_loss(x) increases as well,
  #   min_loss is np.log(alpha0/alpha1) and achived when x -> -\infinity
  #   max_loss is np.log((1-alpha0)/(1-alpha1)) and achived when x -> \infinity
  # if alpha0 > alpha1:
  #   privacy_loss(x) descreases so inverse_privacy_loss(x) descreases as well,
  #   min_loss is np.log((1-alpha0)/(1-alpha1)) and achived when x -> \infinity
  #   max_loss is np.log(alpha0/alpha1)  and achived when x -> -\infinity
  # That means that in case of alpha0 > alpha1 we need to carefully inverse
  # when it's needed.

  sigma /= sensitivity # Normalize sigma.

  # Find min anx max loss.
  if alpha0 < alpha1:
    min_loss = np.log(alpha0/alpha1)
    max_loss = np.log((1-alpha0)/(1-alpha1))
  else:
    min_loss = np.log((1-alpha0)/(1-alpha1))
    max_loss = np.log(alpha0/alpha1)
  assert min_loss < max_loss
  # Create an uniform grid (np.linspace) with discretization such that min_loss
  # and max_loss lies between min and max elements of the grid
  min_grid_loss = int(np.floor(min_loss/discretization))
  max_grid_loss = int(np.ceil(max_loss/discretization))
  grid_loss = np.arange(min_grid_loss, max_grid_loss+1)*discretization

  # Inverse loss grid values
  xs = inverse_privacy_loss(grid_loss, alpha0, alpha1, sigma)

  # compute cdf values of the upper distribution.
  cdf_values = norm.cdf(xs, loc=0, scale=sigma)
  if alpha0 < alpha1:
    cdf_values[0], cdf_values[-1] = 0, 1
  else:
    # privacy_loss(x) decreases, so cdf_values decrease as well
    cdf_values[0], cdf_values[-1] = 1, 0
  probs = np.diff(cdf_values)
  if alpha0 > alpha1:
    # privacy_loss(x) is decreasing, so inverse_privacy_loss as well, so
    # cdf_values are decreasing, so we need to invert probabilities.
    probs = -probs
  # Create Privacy Bucket PLD
  return pld_pmf.DensePLDPmf(discretization, min_grid_loss+1, probs, infinity_mass=0, pessimistic_estimate=True)

def create_pld(discretization:float, alpha0: float, alpha1: float, sigma: float, sensitivity:float = 1):
  pld_pmf1 = create_pld_pmf(discretization, alpha0, alpha1, sigma, sensitivity)
  pld_pmf2 = create_pld_pmf(discretization, alpha1, alpha0, sigma, sensitivity)
  return privacy_loss_distribution.PrivacyLossDistribution(pld_pmf1, pld_pmf2)


In [ ]:
# Check how much better than naive composition
ALPHA0, ALPHA1 = 0.4, 0.6
SIGMA = 3.5
DELTA = 1e-10
N_MECHANISMS = [1, 2, 5, 10, 100, 1000]
DISCRETIZATION = 1e-3 # it controls accuracy/performance tradeoff

pld = create_pld(DISCRETIZATION, ALPHA0, ALPHA1, SIGMA)
print(f"{DELTA=} ")
for n in N_MECHANISMS:
  print(f"{n} mechanisms: eps =", pld.self_compose(n).get_epsilon_for_delta(DELTA))

1 mechanisms: eps= 0.26915235911588586
2 mechanisms: eps= 0.4222320497012769
5 mechanisms: eps= 0.7296331004548241
10 mechanisms: eps= 1.086537633545073
100 mechanisms: eps= 4.1825334968537815
1000 mechanisms: eps= 19.35224047311126


<ipython-input-4-dbe3b5af0cc9>:12: RuntimeWarning: invalid value encountered in log
  return 0.5 + sigma**2*np.log(nom/den)
